# Simple Investment

Load libraries.

In [ ]:
from pyomo.environ import *
from brooks import *
import pandas as pd

Read data with pandas.

In [ ]:
simple_inv_data = pd.read_excel("optimization_data_for_python.xlsx",
                               sheet_name="SimplInv",
                               index_col=0)
L = list(simple_inv_data)
interest = dict(zip(L, simple_inv_data.loc['Interest_rate']))
bad = dict(zip(L, simple_inv_data.loc['Bad_debt_ratio']))
total_limit = 12
pct_FC = 0.4
pct_H = 0.5
pct_bad = 0.04

Declare a model object.

In [ ]:
model = Model()

Declare nonnegative variables for each loan type.

In [ ]:
if hasattr(model, 'x'):
    model.delete(model.x)
model.x = Var(L, domain=NonNegativeReals)

Define the objective function.

In [ ]:
if hasattr(model, 'profit_objective'):
    model.delete(model.profit_objective)
model.profit_objective = (
    Objective(expr = sum(interest[i]*(1-bad[i])*model.x[i] - bad[i]*model.x[i] for i in L), 
              sense=maximize
             )
)

Specify the budget constraint. 

In [ ]:
if hasattr(model, 'total_investment'):
    model.delete(model.total_investment)
model.total_investment = Constraint(expr = sum(model.x[i] for i in L) <= total_limit)

Specify the constraint on farm and commercial investment.

In [ ]:
if hasattr(model, 'fc_constraint'):
    model.delete(model.fc_constraint)
model.fc_constraint = (
    Constraint(expr = model.x['Farm'] + model.x['Commercial'] 
                        >= pct_FC * sum(model.x[i] for i in L) 
              )
)

Specify the constraint on home investment as a percentage of personal, car, and home investment.

In [ ]:
if hasattr(model, 'h_constraint'):
    model.delete(model.h_constraint)
model.h_constraint = (
    Constraint(expr = model.x['Home'] 
                        >= pct_H * (model.x['Personal'] 
                                 + model.x['Car'] 
                                 + model.x['Home'])
              )
)

Specify constraint on bad debt.

In [ ]:
if hasattr(model, 'bad_constraint'):
    model.delete(model.bad_constraint)
model.bad_constraint = (
    Constraint(expr = sum(bad[i]*model.x[i] for i in L)
                        <= pct_bad * sum(model.x[i] for i in L)
              )
)

Check 'written-out' formulation.

In [ ]:
model.pprint()

In [ ]:
solver = SolverFactory('glpk')
status = solver.solve(model)

Get status.

In [ ]:
print('Status = %s' % status)

Get solution and objective function value.

In [ ]:
for i in L:
    print('%s = %f' % (model.x[i], value(model.x[i])))
print("Objective = %f" % value(model.profit_objective))

The optimal solution is to invest \\$7.2 million in home and \\$4.8 million in commercial for a profit of \\$996,480.